In [49]:
import smartsheet
import os
import csv
import pandas as pd
import numpy as np
from socrata_replace.socrata_py_replace import ReplaceBot
from secret_data import userpass
from datetime import date, timedelta, datetime
import holidays as pyholidays
import pytz
from datetime import date, datetime, timedelta
import holidays as pyholidays 


SMARTSHEET_ACCESS_TOKEN = "74xq6QRJHXtN4VdRbwCl9rJeznxjgBcKEPJeJ"
try:
    smartsheet_client = smartsheet.Smartsheet(SMARTSHEET_ACCESS_TOKEN)
except Exception as e:
    print(f"Details: {e}")
    exit()

def get_sheet_data(sheet_id):
    """
    Retrieves the full row and cell data for a specific sheet ID and returns it as a pandas DataFrame.
    Modified to correctly aggregate row data and skip empty rows.
    """
    try:
        sheet_object = smartsheet_client.Sheets.get_sheet(sheet_id)
    except smartsheet.exceptions.ApiError as e:
        print(e.error.message)
        return pd.DataFrame()
    print(f"\n--- Processing Sheet: {sheet_object.name} (ID: {sheet_id}) ---")

    # 1. Get Column Mapping 
    column_map = {col.id: col.title for col in sheet_object.columns}
    column_titles = list(column_map.values())
        
    all_rows_data = []
        
    for row in sheet_object.rows:
        row_data = {}
        has_data = False
        
        for cell in row.cells:
            column_title = column_map.get(cell.column_id, "Unknown Column")
                
            # Check for a display value first, then fallback to the raw value
            value = cell.display_value if cell.display_value is not None else cell.value
                
            row_data[column_title] = value
            
            # Check if the cell has a value to skip entirely blank rows
            if value is not None and value != '':
                has_data = True
                
        if has_data:
            all_rows_data.append(row_data)
        
    print(f"Total NON-BLANK rows added to DataFrame: {len(all_rows_data)}")
        
    df = pd.DataFrame(all_rows_data, columns=column_titles)
        
    df['Sheet Name'] = sheet_object.name
    df['Sheet ID'] = sheet_id
    return df

In [51]:
def get_sheet_id_by_name(sheet_name):
    """
    Finds the ID of a sheet given its name by listing all accessible sheets.
    Returns the Sheet ID (integer) or None if not found.
    """
    if not sheet_name: return None
    
    try:
        response = smartsheet_client.Sheets.list_sheets(include_all=True)
        for sheet in response.data:
            if sheet.name.strip() == sheet_name.strip(): # Use .strip() for robust matching
                return sheet.id
    except smartsheet.exceptions.ApiError as e:
        print(f"API Error finding sheet ID for '{sheet_name}': {e.error.message}")
    return None

In [53]:
all_dfs = []
csv_name = 'Sheet Names.csv'
df_names = pd.read_csv(csv_name)
output_file_name = 'compiled_smartsheet_data.csv'

for names in df_names['Sheet Name']:
    sheet_id = get_sheet_id_by_name(names)
    if sheet_id:
        current_df = get_sheet_data(sheet_id)
        if not current_df.empty:
            all_dfs.append(current_df)
if all_dfs:
    final_compiled_df = pd.concat(all_dfs, ignore_index=True, sort=False)
    final_compiled_df.fillna('', inplace=True)
    final_compiled_df['Duration'] = final_compiled_df['Duration'].replace('', np.nan)
    
    final_compiled_df['Duration_Cleaned'] = final_compiled_df['Duration'].str.replace('d', '', regex=False)
    final_compiled_df['Duration_Days'] = pd.to_numeric(final_compiled_df['Duration_Cleaned'], errors='coerce')
    final_compiled_df['Duration_Days'] = final_compiled_df['Duration_Days'].astype('Int64')
    final_compiled_df['Duration']= final_compiled_df['Duration_Days']
    final_compiled_df = final_compiled_df.drop(['Duration_Days', 'Duration_Cleaned'], axis=1)
    
    print("\n" + "="*50)
    print(f"✅ Compilation Complete! Total Sheets Compiled: {len(all_dfs)}")
    print(f"Final DataFrame shape: {final_compiled_df.shape}")
    print(f"✅ Successfully saved DataFrame to '{output_file_name}'")
    print("="*50)
else:
    print("\n--- No sheets were successfully compiled. ---")


# The Eastern Timezone name for pytz, which handles both EST and EDT (Daylight Saving)
TARGET_TZ = 'US/Eastern'

date_columns = [
    'Submitted Date',
    'Communication Start Date',
    'Initial Assessment Date',
    'Review Start Date',
    'Final Resolution Date',
    'Expiration Date'
]

for col in date_columns:
    print(f"\nProcessing column: {col}")
   
    final_compiled_df[col] = pd.to_datetime(
        final_compiled_df[col],
        errors='coerce',
        utc=True 
    )
    if not pd.api.types.is_datetime64_any_dtype(final_compiled_df[col]):
        print(f"Column '{col}' failed to convert to a datetime dtype or is missing a value. Skipping.")
        final_compiled_df[col] = np.nan
        continue

    final_compiled_df[col] = final_compiled_df[col].dt.tz_convert(TARGET_TZ)
        
    date_only_series = final_compiled_df[col].dt.date.apply(lambda x: x.isoformat() if pd.notna(x) else np.nan)

    final_compiled_df[col] = final_compiled_df[col].apply(
        lambda x: x.isoformat() if pd.notna(x) else np.nan
    )
final_compiled_df.to_csv(output_file_name, index=False, encoding='utf-8')
print("\nScript finished.")


--- Processing Sheet: Bainbridge Permit Tracking Sheet (ID: 595984821211012) ---
Total NON-BLANK rows added to DataFrame: 20

--- Processing Sheet: Burnt Hill Farm Permit Tracking Sheet (ID: 434783269113732) ---
Total NON-BLANK rows added to DataFrame: 0

--- Processing Sheet: Loch Raven Permit Tracking Sheet (ID: 8555094548107140) ---
Total NON-BLANK rows added to DataFrame: 2

--- Processing Sheet: Pimlico Permit Tracking Sheet (ID: 8603266901299076) ---
Total NON-BLANK rows added to DataFrame: 12

--- Processing Sheet: Quantum Permit Tracking Sheet (ID: 2274508036591492) ---
Total NON-BLANK rows added to DataFrame: 28

--- Processing Sheet: SBY Market Permit Tracking Sheet (ID: 6312015061536644) ---
Total NON-BLANK rows added to DataFrame: 9

✅ Compilation Complete! Total Sheets Compiled: 5
Final DataFrame shape: (71, 31)
✅ Successfully saved DataFrame to 'compiled_smartsheet_data.csv'

Processing column: Submitted Date

Processing column: Communication Start Date

Processing colum

In [57]:
portal = "data.maryland.gov"
uid='b4mc-xw4p'
rb = ReplaceBot(False, data_folder="", filename=output_file_name, socrata_server=portal)
rb.upload_replacement(uid, 'private', filename=output_file_name)

True

In [81]:
from datetime import date, datetime, timedelta
import holidays as pyholidays # Assuming you are using the 'holidays' library

def count_maryland_business_days(start_date, end_date):
    """
    Calculates the total number of business days (Mon-Fri, excluding 
    US federal and Maryland state holidays) between a start_date and an end_date (inclusive of end_date).

    :param start_date: The date to start the calculation from (datetime.date object).
    :param end_date: The date to end the calculation at (datetime.date object).
    :return: The number of business days in the range.
    """
    # NOTE: Assuming start_date and end_date are already datetime.date objects 
    # as per the DataFrame preparation below.

    # Ensure start and end dates are valid for calculation
    if not (isinstance(start_date, date) and isinstance(end_date, date)):
        return np.nan
    
    # Handle NaT values passed from pandas
    if pd.isna(start_date) or pd.isna(end_date):
        return np.nan

    # If the process isn't complete, return 0 or NaN
    if start_date > end_date:
        return 0

    # 1. Get the list of Maryland holidays within the required year range
    md_holidays = pyholidays.US(state='MD', years=range(start_date.year, end_date.year + 2))

    business_day_count = 0
    current_date = start_date

    # 2. Loop through each day from start_date up to and including end_date
    while current_date <= end_date:
        
        # Check 1: Is the current day a weekend day? (Saturday=5, Sunday=6)
        is_weekend = current_date.weekday() >= 5
        
        # Check 2: Is the current day a holiday?
        is_holiday = current_date in md_holidays
        
        # Check 3: If it's NOT a weekend AND NOT a holiday, it's a business day.
        if not is_weekend and not is_holiday:
            business_day_count += 1
        
        # Move to the next day
        current_date += timedelta(days=1)

    return business_day_count

In [99]:
import pandas as pd
import numpy as np
from datetime import date # date.today() comes from here

# Assuming date_columns is defined and ALL dates have been successfully 
# converted to timezone-aware Pandas Datetime objects (e.g., Eastern Time).

# --- 1. Define the List of START Dates for the calculation ---
# You want the calculation for ALL columns listed, up to today.
start_date_columns = [
    'Submitted Date',
    'Communication Start Date',
    'Initial Assessment Date',
    'Review Start Date',
    'Final Resolution Date',
    'Expiration Date' 
    # NOTE: You must include only columns that make sense as START dates
]

print("Starting business day calculations against today's date...")

# --- 2. Loop and Apply the Custom Function (axis=1 is crucial) ---

for start_col in start_date_columns:
    
    # 1. Define the new column name
    # Example: 'Submitted Date' -> 'submitted_date_business_days_to_today'
    new_col_name = start_col.replace(' Date', '').replace(' ', '_').lower() + "_business_days_to_today"
    
    # 2. Use .apply(..., axis=1) to pass the start date for each row
    final_compiled_df[new_col_name] = final_compiled_df.apply(
        lambda row: count_maryland_business_days(
            # Pass the START DATE: The date from the current row/column.
            # We use .date() to convert the Pandas Datetime object to a Python date object.
            row[start_col].date(), 
            # Pass the END DATE: Today's fixed date
            date.today()
        )
        # Use a quick check for NaT values before calling the function
        if pd.notna(row[start_col]) else np.nan, 
        axis=1
    )
    
    print(f"✅ Calculated '{new_col_name}'.")

print("\nBusiness day calculation loop complete.")

Starting business day calculations against today's date...


AttributeError: 'str' object has no attribute 'date'